In [121]:
import pandas as pd
import numpy as np
import psycopg2
from sklearn.preprocessing import MinMaxScaler

In [122]:
def create_connection():
    conn = psycopg2.connect(
        user=login['user'],
        password=login['password'],
        host=login['host'],
        port=login['port'],
        database=login['database']
    )
    print("PostgreSQL DB에 연결되었습니다.")

    return conn

In [123]:
# SQL 쿼리
sql = """
SELECT 	cp.price 
	,	pg.grade 
	,	cp.favoritecount 
	,	cp.viewcount 
	,	p2.screen_size 
	,	pcc."cost" 
	,	pcc.capacity 
	,	p2.ram 
	,	p2.weight 
	,	p2.max_video_playtime 
	,	p2.max_audio_playtime 
	,	p.battery_capacity 
	,	cp.hours 
    ,	p.release_year 
    ,	cp.crawling
FROM product.crwaling_phone cp 
LEFT JOIN product.product_grade pg 
ON cp.grade_id = pg.grade_id 
LEFT JOIN product.product p 
ON pg.product_id = p.id
LEFT JOIN product.phone_capacity_cost pcc 
ON cp.phone_cost_id = pcc.id 
LEFT JOIN product.phone p2 
ON pcc.product_id = p2.product_id 
WHERE 	cp.price IS NOT NULL
	AND pg.grade IS NOT NULL
	AND cp.favoritecount IS NOT NULL
	AND cp.viewcount IS NOT NULL
	AND p2.screen_size IS NOT NULL
	AND pcc."cost" IS NOT NULL
	AND pcc.capacity IS NOT NULL
	AND p2.ram IS NOT NULL
	AND p2.weight IS NOT NULL
	AND p2.max_video_playtime IS NOT NULL
	AND p2.max_audio_playtime IS NOT NULL
	AND p.battery_capacity IS NOT NULL
	AND cp.hours IS NOT NULL;
"""

# PostgreSQL 연결 생성
conn = create_connection()

# SQL 쿼리 실행 후 DataFrame으로 저장
df = pd.read_sql_query(sql, conn)

# 연결 종료
conn.close()


PostgreSQL DB에 연결되었습니다.


C:\Users\영호\AppData\Local\Temp\ipykernel_29604\3593964391.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


In [119]:
df

,price,grade,favoritecount,viewcount,screen_size,cost,capacity,ram,weight,max_video_playtime,max_audio_playtime,battery_capacity,hours,release_year,crawling
0,970000,C,0,0,19.21,1998700,256,12,0.263,20,72,4400.0,0.016666666666666666,2022,2
1,970000,C,0,0,19.21,1998700,256,12,0.263,20,72,4400.0,0.016666666666666666,2022,2
2,970000,C,0,0,19.21,1998700,256,12,0.263,20,72,4400.0,0.016666666666666666,2022,2
3,440000,C,0,0,15.74,1248500,128,12,0.163,20,79,4000.0,0.1,2020,1
4,420000,A,0,1,17.45,1452000,256,12,0.208,23,103,4500.0,0.13333333333333333,2020,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11291,35000,A,1,77,17.01,1650000,256,8,0.183,18,40,3300.0,2.0,2020,2
11292,20000,C,2,67,17.01,1650000,256,8,0.183,18,40,3300.0,3.0,2020,2
11293,100000,C,0,41,17.01,1650000,256,8,0.183,18,40,3300.0,5.0,2020,2
11294,23000,C,0,35,17.01,1650000,256,8,0.183,18,40,3300.0,8.0,2020,2


In [124]:
def convert_as_used_year(df):
    '''
    2023년 대비 년식을 구하는 함수
    '''
    df = df.fillna(0)
    df['used_year'] = df['release_year'].apply(lambda x: 2023 - x)

    return df

In [125]:
df = convert_as_used_year(df)

In [92]:
df['hours']

0        0.016666666666666666
1        0.016666666666666666
2        0.016666666666666666
3                         0.1
4         0.13333333333333333
                 ...         
11291                     2.0
11292                     3.0
11293                     5.0
11294                     8.0
11295                     8.0
Name: hours, Length: 11296, dtype: object

In [126]:
def modeling_Preprocessor(df):
    # 로그 변환 전 준비
    # 0인 데이터에 0.001 추가하기 (최대한 데이터 손상을 막기 위해)
    columns_to_update = ['favoritecount', 'viewcount', 'used_year']
    
    # 각 열에 0.001 더하기
    for column in columns_to_update:
        df[column] = df[column].apply(lambda x: x + 0.001)
    return df  # DataFrame 반환



In [127]:
df = modeling_Preprocessor(df)

In [103]:
df['hours']

0        0.016666666666666666
1        0.016666666666666666
2        0.016666666666666666
3                         0.1
4         0.13333333333333333
                 ...         
11291                     2.0
11292                     3.0
11293                     5.0
11294                     8.0
11295                     8.0
Name: hours, Length: 11296, dtype: object

In [128]:
def log_transform(df):
    '''
    로그 변환이 적용된 데이터프레임을 반환하는 함수
    price 종속변수 제외
    '''
    # 수치형 데이터만 선택
    num_cols = df.select_dtypes(include=[np.number]).columns

    for column in num_cols:
        if column not in ['price']:
            filtered = df[column][df[column] > 0] 
            df[column] = np.log(filtered) 
    
    # price 컬럼을 int로 변환
    df['price'] = df['price'].astype(int)
    
    return df  # DataFrame 반환


In [129]:
df = log_transform(df)

In [106]:
df['hours']

0        0.016666666666666666
1        0.016666666666666666
2        0.016666666666666666
3                         0.1
4         0.13333333333333333
                 ...         
11291                     2.0
11292                     3.0
11293                     5.0
11294                     8.0
11295                     8.0
Name: hours, Length: 11296, dtype: object

In [130]:
def get_minmax_scaler(df, exclude_columns=['price']):
    '''
    price(종속변수) 열을 제외한 스케일링 함수
    '''
    scaler = MinMaxScaler()
    scaled_df = df.copy()

    for column in df.select_dtypes(include=[np.number]).columns:
        if column not in exclude_columns: 
            scaled_values = scaler.fit_transform(scaled_df[[column]])  
            scaled_df[column] = scaled_values  
    return scaled_df  # DataFrame 반환


In [131]:
df = get_minmax_scaler(df)


In [112]:
df

,price,grade,favoritecount,viewcount,screen_size,cost,capacity,ram,weight,max_video_playtime,max_audio_playtime,battery_capacity,hours,release_year,used_year
0,970000,C,NaN,NaN,0.990334,0.876305,256,1.000000,0.896215,0.804163,0.769301,0.873439,0.016666666666666666,1.000000,0.000000
1,970000,C,NaN,NaN,0.990334,0.876305,256,1.000000,0.896215,0.804163,0.769301,0.873439,0.016666666666666666,1.000000,0.000000
2,970000,C,NaN,NaN,0.990334,0.876305,256,1.000000,0.896215,0.804163,0.769301,0.873439,0.016666666666666666,1.000000,0.000000
3,440000,C,NaN,NaN,0.578342,0.556737,128,1.000000,0.184404,0.804163,0.818750,0.779077,0.1,0.333498,0.792481
4,420000,A,NaN,0.000000,0.791620,0.659285,256,1.000000,0.547133,0.926822,0.960134,0.895688,0.13333333333333333,0.333498,0.792481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11291,35000,A,0.000000,0.414807,0.738808,0.746101,256,0.707519,0.356606,0.711695,0.456035,0.588620,2.0,0.333498,0.792481
11292,20000,C,0.112579,0.401523,0.738808,0.746101,256,0.707519,0.356606,0.711695,0.456035,0.588620,3.0,0.333498,0.792481
11293,100000,C,NaN,0.354624,0.738808,0.746101,256,0.707519,0.356606,0.711695,0.456035,0.588620,5.0,0.333498,0.792481
11294,23000,C,NaN,0.339514,0.738808,0.746101,256,0.707519,0.356606,0.711695,0.456035,0.588620,8.0,0.333498,0.792481


In [87]:
df['hours']

KeyError: 'hours'

In [132]:
# 등급 변환
def extract_and_map_quality(df):
    # 등급 점수화
    quality_mapping = {
        '미개봉': 5,
        'S급': 4,
        'A급': 3,
        'B급': 2,
        'C급': 1,
    }
    df['grade'] = df['grade'].map(quality_mapping)
    return df

# 조회수, 찜(좋아요) 가중치
def normalize_viewcount_favoritecount(df):
    '''
    웹페이지 사용자 수를 기준으로 'viewcount'와 'favoritecount' 열을 정규화하는 함수
    '''
    user_site = {
        0: 4640000,  # 당근마켓
        2: 490000,   # 번개장터
        1: 120000,   # 중고나라
    }

    for site, user_count in user_site.items():
        df.loc[df['crawling'] == site, 'viewcount'] /= user_count
        df.loc[df['crawling'] == site, 'favoritecount'] /= user_count

    # nan 값은 0으로 변환
    df['viewcount'].fillna(0, inplace=True)
    df['favoritecount'].fillna(0, inplace=True)
    return df


In [133]:
df = extract_and_map_quality(df)

In [134]:
df = normalize_viewcount_favoritecount(df)